In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
company_file = "/ml_insurance_challenge.csv"
taxonomy_file = "/insurance_taxonomy.xlsx"

data_companies = pd.read_csv(company_file)
data_taxonomy = pd.read_excel(taxonomy_file)

data_companies.columns = data_companies.columns.str.lower().str.strip()
data_taxonomy.columns = data_taxonomy.columns.str.lower().str.strip()

company_columns = ["description", "business_tags", "sector", "category", "niche"]
taxonomy_column = "label"

combined_text = []
for _, row in data_companies.iterrows():
  row_text = []
  for col in company_columns:
    row_text.append(str(row[col]))
  combined_text.append(' '.join(row_text))

data_taxonomy[taxonomy_column] = [str(elem) for elem in data_taxonomy[taxonomy_column]]

model = SentenceTransformer('all-MiniLM-L6-v2')

company_embeddings = model.encode(list(combined_text), convert_to_tensor=True)
taxonomy_embeddings = model.encode(list(data_taxonomy[taxonomy_column]), convert_to_tensor=True)

similarity_matrix = cosine_similarity(company_embeddings.cpu().numpy(), taxonomy_embeddings.cpu().numpy())

classified_labels = []
for row in similarity_matrix:
  matched_labels = []
  threshold = 0.99
  while matched_labels == [] and threshold > 0:
    matched_labels = [data_taxonomy[taxonomy_column].iloc[i] for i in np.where(row >= threshold)[0]]
    threshold -= 0.01
  classified_labels.append(', '.join(matched_labels) if matched_labels else "Unclassified")

data_companies["insurance_label"] = classified_labels

data_companies.to_csv("/classified_companies.csv", index=False)

print("Classification completed! Results saved in 'classified_companies.csv'.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Classification completed! Results saved in 'classified_companies.csv'.
